# Leer el archivo

In [2]:
import pandas as pd

# Cargar el CSV
csv_path = 'data/export.csv'
df = pd.read_csv(csv_path, encoding='utf-16', sep='\t')

# 1. Ver columnas del CSV
print(f"📊 El CSV tiene {len(df.columns)} columnas.")
print("📝 Columnas del CSV:", list(df.columns))

# 2. Mostrar los primeros registros para ver el formato
df.head()


📊 El CSV tiene 74 columnas.
📝 Columnas del CSV: ['sequence_id ', 'razon_social                                                                                        ', 'nombre_comercial                                                                     ', 'sector                                           ', 'actividad_2                                                                                                                                                                                                                                                                                                             ', 'actividad                                                                                                                                                                                                                                                                                                               ', 'cif            ', 'direccion_del_negocio                

/var/folders/09/3xcrkv750gd3cy0wyd9255rm0000gn/T/ipykernel_13465/1629525645.py:5: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='utf-16', sep='\t')


,sequence_id,razon_social,nombre_comercial,sector,actividad_2,actividad,cif,direccion_del_negocio,codigo_postal,localidad,...,facturacion_ordenar,resultado_2023,url_edicion_impresa,capital_social,username,fee_id,terms_and_conditions_acceptance_date,expires_on,created_date,modified_date
0,1,Iteisa Desarrollo y Sistemas SL ...,...,Informática/Nuevas tecnologías,Desarrollo de aplicaciones informáticas ...,Desarrollo de aplicaciones informáticas ...,,Benidorm 8 Bajo ...,39005,,...,,...,,3100,Datos,2,,,"febrero 12, 2025 5:59 pm","febrero 12, 2025 5:59 pm"
1,2,KLM Aotearoa SL ...,...,Servicios a la empresa,Asesoramiento y consultoría en el campo económ...,Asesoramiento y consultoría en el campo económ...,,Rentería Reyes 9 Bajo ...,39740,,...,,...,,3000,Datos,2,,,"febrero 12, 2025 5:20 pm","febrero 12, 2025 5:44 pm"
2,3,Peña Miera SRL ...,...,Agricultura/Ganadería,"Actividad agrícola, ganadera y forestal. Comer...","Actividad agrícola, ganadera y forestal. Comer...",,San Fernando s/n ...,39409,,...,,...,,320000,Datos,2,,,"febrero 12, 2025 5:17 pm","febrero 12, 2025 5:17 pm"
3,4,APJ Wind SL ...,...,Banca/Servicios financieros/Seguros,Participación en cualquier tipo de sociedades ...,Participación en cualquier tipo de sociedades ...,,Barrio Rubo s/n ...,39478,Boo,...,,...,,3100,Datos,,,,"febrero 12, 2025 5:15 pm","febrero 12, 2025 5:15 pm"
4,5,Gilda y Asociados SL ...,...,Banca/Servicios financieros/Seguros,Actividades auxiliares a los servicios financi...,Actividades auxiliares a los servicios financi...,,Camino Calero-Rionegro 400 ...,39795,,...,,...,,3000,Datos,2,,,"febrero 12, 2025 5:13 pm","febrero 12, 2025 5:13 pm"


In [3]:
import pandas as pd
from datetime import datetime

# 📌 1️⃣ Cargar el CSV
csv_path = "data/export.csv"
df = pd.read_csv(csv_path, encoding="utf-16", sep="\t", low_memory=False, dtype=str)

# 📌 2️⃣ Eliminar 'sequence_id' si existe (porque es autoincrement en MySQL)
if "sequence_id" in df.columns:
    df.drop(columns=["sequence_id"], inplace=True)

# 📌 3️⃣ Reemplazar valores NaN con None para que MySQL los interprete como NULL
df = df.where(pd.notnull(df), None)

# 📌 4️⃣ Diccionario para traducir los meses de español a inglés
meses_abreviados = {
    "ene": "01", "feb": "02", "mar": "03", "abr": "04",
    "may": "05", "jun": "06", "jul": "07", "ago": "08",
    "sep": "09", "oct": "10", "nov": "11", "dic": "12"
}

def convertir_fecha_fundacion(fecha):
    """Convierte fechas en el campo 'fecha_de_fundacion_de_la_empresa' al formato MySQL (YYYY-MM-DD)"""
    if fecha and isinstance(fecha, str):
        fecha = fecha.strip()
        try:
            # 📌 Si la fecha es solo el año (ej: "2024") → convertir a "2024-01-01"
            if fecha.isdigit() and len(fecha) == 4:
                return f"{fecha}-01-01"
            
            # 📌 Si la fecha es formato abreviado (ej: "jul-22") → convertir a "2022-07-01"
            if "-" in fecha:
                mes, anio = fecha.split("-")
                mes = mes.lower()
                if mes in meses_abreviados and anio.isdigit():
                    return f"20{anio}-{meses_abreviados[mes]}-01"

            # Si el formato es incorrecto, devolver None
            return None
        except ValueError:
            return None  
    return None  

# 📌 5️⃣ Aplicar conversión de fechas en 'fecha_de_fundacion_de_la_empresa'
if "fecha_de_fundacion_de_la_empresa" in df.columns:
    df["fecha_de_fundacion_de_la_empresa"] = df["fecha_de_fundacion_de_la_empresa"].apply(convertir_fecha_fundacion)

# 📌 6️⃣ Aplicar conversión de fechas para otros campos
meses_espanol = {
    "enero": "January", "febrero": "February", "marzo": "March", "abril": "April",
    "mayo": "May", "junio": "June", "julio": "July", "agosto": "August",
    "septiembre": "September", "octubre": "October", "noviembre": "November", "diciembre": "December"
}

def convertir_fecha(fecha):
    """Convierte fechas en español al formato MySQL (YYYY-MM-DD HH:MM:SS)"""
    if fecha and isinstance(fecha, str):
        try:
            for mes_es, mes_en in meses_espanol.items():
                if mes_es in fecha.lower():
                    fecha = fecha.lower().replace(mes_es, mes_en)
                    break
            return datetime.strptime(fecha, "%B %d, %Y %I:%M %p").strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            return None  
    return None  

# Aplicar conversión de fechas en 'created_date' y 'modified_date'
columnas_fecha = ["created_date", "modified_date"]
for col in columnas_fecha:
    if col in df.columns:
        df[col] = df[col].apply(convertir_fecha)

# 📌 7️⃣ Función para limpiar valores DECIMAL
def limpiar_decimal(valor):
    if valor is None or pd.isna(valor) or valor == "":
        return None  # Convertir valores vacíos a None
    if isinstance(valor, (float, int)):
        return round(valor, 2) if abs(valor) <= 999999999999.99 else None
    try:
        valor = valor.replace(",", ".")  # Reemplazar coma decimal por punto (si existe)
        valor = "".join(c for c in valor if c.isdigit() or c == ".")  # Mantener solo números y punto decimal
        valor_float = float(valor)
        return round(valor_float, 2) if abs(valor_float) <= 999999999999.99 else None
    except ValueError:
        return None  

# 📌 8️⃣ Aplicar la función a las columnas DECIMAL
columnas_decimales = [
    "facturacion_2004", "facturacion_2005", "facturacion_2006", "facturacion_2007",
    "facturacion_2008", "facturacion_2009", "facturacion_2010", "facturacion_2011",
    "facturacion_2012", "facturacion_2013", "facturacion_2014", "facturacion_2015",
    "facturacion_2016", "facturacion_2017", "resultado_2017", "facturacion_2018",
    "resultado_2018", "facturacion_2019", "resultado_2019", "facturacion_2020",
    "resultado_2020", "facturacion_2021", "resultado_2021", "facturacion_2022",
    "resultado_2022", "facturacion_2023", "resultado_2023", "facturacion_2024",
    "resultado_2024", "facturacion_ordenar", "capital_social"
]

for col in columnas_decimales:
    if col in df.columns:
        df[col] = df[col].apply(limpiar_decimal)

# 📌 9️⃣ Guardar el nuevo CSV corregido listo para MySQL
nuevo_csv_path = "data/export_limpio.csv"
df.to_csv(nuevo_csv_path, encoding="utf-8", sep=",", index=False)

print(f"✅ Archivo export_limpio.csv guardado en: {nuevo_csv_path}")


✅ Archivo export_limpio.csv guardado en: data/export_limpio.csv


In [5]:
import pandas as pd

# 📌 Ruta del CSV corregido
csv_path = "data/export_limpio.csv"

# 📌 Leer el archivo con UTF-8 y separador correcto ","
df = pd.read_csv(csv_path, encoding="utf-8", sep=",")

# 📌 Verificar columnas y primeros registros
print(f"📊 El CSV tiene {len(df.columns)} columnas.")
print("📝 Columnas del CSV:", list(df.columns))
df.head()


📊 El CSV tiene 74 columnas.
📝 Columnas del CSV: ['sequence_id ', 'razon_social                                                                                        ', 'nombre_comercial                                                                     ', 'sector                                           ', 'actividad_2                                                                                                                                                                                                                                                                                                             ', 'actividad                                                                                                                                                                                                                                                                                                               ', 'cif            ', 'direccion_del_negocio                

/var/folders/09/3xcrkv750gd3cy0wyd9255rm0000gn/T/ipykernel_13465/3461803544.py:7: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding="utf-8", sep=",")


,sequence_id,razon_social,nombre_comercial,sector,actividad_2,actividad,cif,direccion_del_negocio,codigo_postal,localidad,...,facturacion_ordenar,resultado_2023,url_edicion_impresa,capital_social,username,fee_id,terms_and_conditions_acceptance_date,expires_on,created_date,modified_date
0,1,Iteisa Desarrollo y Sistemas SL ...,...,Informática/Nuevas tecnologías,Desarrollo de aplicaciones informáticas ...,Desarrollo de aplicaciones informáticas ...,,Benidorm 8 Bajo ...,39005,,...,,...,,3100,Datos,2,,,"febrero 12, 2025 5:59 pm",2025-02-12 17:59:00
1,2,KLM Aotearoa SL ...,...,Servicios a la empresa,Asesoramiento y consultoría en el campo económ...,Asesoramiento y consultoría en el campo económ...,,Rentería Reyes 9 Bajo ...,39740,,...,,...,,3000,Datos,2,,,"febrero 12, 2025 5:20 pm",2025-02-12 17:44:00
2,3,Peña Miera SRL ...,...,Agricultura/Ganadería,"Actividad agrícola, ganadera y forestal. Comer...","Actividad agrícola, ganadera y forestal. Comer...",,San Fernando s/n ...,39409,,...,,...,,320000,Datos,2,,,"febrero 12, 2025 5:17 pm",2025-02-12 17:17:00
3,4,APJ Wind SL ...,...,Banca/Servicios financieros/Seguros,Participación en cualquier tipo de sociedades ...,Participación en cualquier tipo de sociedades ...,,Barrio Rubo s/n ...,39478,Boo,...,,...,,3100,Datos,,,,"febrero 12, 2025 5:15 pm",2025-02-12 17:15:00
4,5,Gilda y Asociados SL ...,...,Banca/Servicios financieros/Seguros,Actividades auxiliares a los servicios financi...,Actividades auxiliares a los servicios financi...,,Camino Calero-Rionegro 400 ...,39795,,...,,...,,3000,Datos,2,,,"febrero 12, 2025 5:13 pm",2025-02-12 17:13:00


In [7]:
import pandas as pd
import mysql.connector
from datetime import datetime

# 📌 1️⃣ Cargar el CSV corregido
csv_path = "data/export_limpio.csv"
df = pd.read_csv(csv_path, encoding="utf-8", sep=",", low_memory=False, dtype=str)
import pandas as pd
import mysql.connector

df = df.where(pd.notnull(df), None)  # Manejar valores nulos

# 📌 2️⃣ Renombrar la columna incorrecta
if "director__gerente" in df.columns:
    df.rename(columns={"director__gerente": "director_gerente"}, inplace=True)

# Conexión a MySQL
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='test_directorios'
)
cursor = conexion.cursor()

# Consulta SQL (ahora sin 'sequence_id')
consulta = """
INSERT INTO pruebaDirectorio (
    razon_social, nombre_comercial, sector, actividad_2, actividad, cif,
    direccion_del_negocio, codigo_postal, localidad, municipio,
    sitio_web_del_negocio, fax_del_negocio, numero_telefonico_del_negocio,
    descripcion_corta_del_negocio, fecha_de_fundacion_de_la_empresa,
    correo_electronico_de_contacto_del_negocio, poligono_industrial,
    facturacion_2004, facturacion_2005, facturacion_2006, facturacion_2007,
    facturacion_2008, facturacion_2009, facturacion_2010, facturacion_2011,
    facturacion_2012, facturacion_2013, facturacion_2014, facturacion_2015,
    facturacion_2016, facturacion_2017, resultado_2017, facturacion_2018,
    resultado_2018, facturacion_2019, resultado_2019, facturacion_2020,
    resultado_2020, facturacion_2021, resultado_2021, director_gerente,
    cargo_1, cargo_2, cargo_3, cargo_4, cargo_5, cargo_6, cargo_7, cargo_8,
    cargo_9, numero_de_trabajadores, exporta, que_productos_exporta,
    estado_de_la_sociedad, sector_ordenar, no_trabajadores_ordenar,
    ver_facturaciones_anteriores_a_2018, municipio_ordenar, resultado_2024,
    facturacion_2024, resultado_2022, facturacion_2023, facturacion_2022,
    facturacion_ordenar, resultado_2023, url_edicion_impresa, capital_social,
    username, fee_id, terms_and_conditions_acceptance_date, expires_on,
    created_date, modified_date
)
VALUES ({});
""".format(",".join(["%s"] * 73))  # Generar 73 placeholders dinámicamente

# Insertar filas
for _, fila in df.iterrows():
    valores = tuple(fila)
    cursor.execute(consulta, valores)

# Confirmar cambios y cerrar la conexión
conexion.commit()
cursor.close()
conexion.close()

print("✅ ¡Datos importados correctamente a la base de datos!")


ProgrammingError: Not all parameters were used in the SQL statement